In [25]:
from pymongo import MongoClient
import pymongo
import feedparser
from pprint import pprint
import datetime

# open database connection
username="root"
password="mongo"
client = MongoClient('mongodb://%s:%s@127.0.0.1' % (username, password))
db = client.headings
db.headings.create_index([('title', pymongo.TEXT)], name='search_index', default_language='english')

# loop on sources
for source in db.sources.find():

    if "userAgent" in source:
        feed = feedparser.parse(source["rss"], agent=source["userAgent"])
    else:
        feed = feedparser.parse(source["rss"])
    print(feed.href, len(feed.entries))
    
    for entry in feed.entries:
        entry["_source"] = source
        entry["_fetched"] = datetime.datetime.utcnow()
        entry["_timestamp"] = datetime.datetime(*entry["published_parsed"][0:6]) if "published_parsed" in entry else entry["_fetched"]
        
        key = {"id": entry.id}
        db.headings.replace_one(key, entry, True)


https://www.theguardian.com/uk/rss 106
http://feeds.bbci.co.uk/news/uk/rss.xml 40
https://www.dailymail.co.uk/home/index.rss 150
https://www.telegraph.co.uk/rss.xml 100
http://www.independent.co.uk/rss 101
https://www.mirror.co.uk/?service=rss 25
https://www.thesun.co.uk/feed/ 100
http://feeds.feedburner.com/daily-express-news-showbiz 10
https://metro.co.uk/feed/ 30
https://www.channel4.com/news/feed 15
https://www.huffingtonpost.co.uk/feeds/index.xml 50
